In [1]:
import numpy as np
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
from small3DNet import small3DNet
from separateChannel3DNet import separateChannel3DNet
from twoChannel3DNet import twoChannel3DNet
from util import add_color, colorize, colorize_gaussian, calculate_correct_loss
from colorMNist import colorMNist
import random
import colorsys
import pickle
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import wandb

In [ ]:
wandb.init(project="my-test-project", entity="ntahur")

In [ ]:
wandb.finish()

In [2]:
# Load data from pickle file
cmnist_train, cmnist_val, cmnist_test = pickle.load(open("custom_datasets/5k/cmnist_gaussian_18.pkl", "rb"))
print(len(cmnist_train), len(cmnist_val), len(cmnist_train) + len(cmnist_val))

# Create datasets
train_dataset = colorMNist(cmnist_train)
val_dataset = colorMNist(cmnist_val)
test_dataset = colorMNist(cmnist_test)

# Dataloaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
                                               shuffle=False, num_workers = 0)

4000 1000 5000


In [ ]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='datasets/cifar10', train=True, download=False, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='datasets/cifar10', train=False, download=False, transform=transform)

# Create the 80/20 train/val split
train_split = int(0.8 * len(trainset))
# Train dataset
train_dataset = torch.utils.data.Subset(trainset, [i for i in range(train_split)])
# Validation dataset
val_dataset = torch.utils.data.Subset(trainset, [i for i in range(train_split, len(trainset))])
print(len(train_dataset), len(val_dataset), len(train_dataset) + len(val_dataset))

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [3]:
# Layers of the model
# model_layers = [8, 8, "M", 16,"M"]
model_layers = [16, "M", 16, "M", 64,"M"]
# Create model
# Set seed
torch.manual_seed(12)
# Create model
model = small3DNet(model_layers, model_layers[-2])
# model = separateChannel3DNet(model_layers, 16)
# Load file and save file
# lfile = "Gaussian3D_0"
sfile = "Gaussian3D_uniform"
# # Load model
# model.load_state_dict(torch.load('model_saves/new_fair/' + lfile + '.pth'))
# Put model on gpu
model.cuda()
# Loss function
loss_fn = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
# Freeze weights
children = [x for x in model.children()]
for x in children[0]:
    for param in x.parameters():
        param.requires_grad = False

In [4]:
# Number of epochs to train
epochs = 20

# Placeholder variables to put training and validation accuracies and losses per epoch
train_accuracies = []
train_losses = []
val_accuracies = []
val_losses = []

for epoch in tqdm(range(epochs), total=epochs, desc='Training'):
    # print("Epoch", epoch + 1, "/", epochs)
    
    # Update learning rate
    # if (epoch + 1) % 3 == 0:
    #     for g in optimizer.param_groups:
    #         g['lr'] /= 10
    
    # Weights and Biases code
    # children = [x for x in model.children()]
    # for i, x in enumerate(children[0][0].weight):
    #     x0 = x[0][0].cpu().detach().numpy().flatten().reshape(1, -1)
    #     x1 = x[0][1].cpu().detach().numpy().flatten().reshape(1, -1)
    #     x2 = x[0][2].cpu().detach().numpy().flatten().reshape(1, -1)
    #     wandb.log({"Uavgsim-"+str(i): (cosine_similarity(x0, x1)[0][0] + cosine_similarity(x0, x2)[0][0] + cosine_similarity(x1, x2)[0][0]) / 3})
    
    # Put model on training mode
    model.train()
    train_total_correct = 0
    train_total_loss = []
    
    for (images, labels) in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Calculate number correct and loss in batch
        correct, loss = calculate_correct_loss(model, loss_fn, images, labels, model_type=3)
        
        # Backpropagation
        loss.backward()
        # Step function
        optimizer.step()
        
        # Update amount correct and loss with current batch
        train_total_correct += correct
        train_total_loss.append(loss.item())
        
    # Append epoch accuracy and loss
    train_accuracies.append(train_total_correct / len(train_dataset))
    train_losses.append(sum(train_total_loss) / len(train_total_loss))
    
    # Put model on evaluation mode
    model.eval()
    val_total_correct = 0
    val_total_loss = []
    
    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in val_dataloader:
        
            # Calculate number correct and loss in batch
            correct, loss = calculate_correct_loss(model, loss_fn, images, labels, model_type=3)

            # Update amount correct and loss with current batch
            val_total_correct += correct
            val_total_loss.append(loss.item())

    # Append epoch accuracy and loss
    val_accuracies.append(val_total_correct / len(val_dataset))
    val_losses.append(sum(val_total_loss) / len(val_total_loss))
    

# Print accuracies and losses per epoch
for i in range(epochs):
    print("Epoch", i + 1)
    print("Train acc and loss\t", train_accuracies[i], "\t", train_losses[i])
    print("Val acc and loss\t", val_accuracies[i], "\t", val_losses[i])

Training: 100%|██████████| 20/20 [00:20<00:00,  1.01s/it]

Epoch 1
Train acc and loss	 0.35575 	 1.8737903995513916
Val acc and loss	 0.595 	 1.0613581649959087
Epoch 2
Train acc and loss	 0.6775 	 0.794496253490448
Val acc and loss	 0.716 	 0.6889557782560587
Epoch 3
Train acc and loss	 0.75025 	 0.6287787902355194
Val acc and loss	 0.79 	 0.5571339121088386
Epoch 4
Train acc and loss	 0.8135 	 0.49315749621391297
Val acc and loss	 0.847 	 0.44666180247440934
Epoch 5
Train acc and loss	 0.87725 	 0.3537417013645172
Val acc and loss	 0.907 	 0.2736019673757255
Epoch 6
Train acc and loss	 0.90075 	 0.2775509526133537
Val acc and loss	 0.914 	 0.29737630137242377
Epoch 7
Train acc and loss	 0.921 	 0.22388241267204284
Val acc and loss	 0.931 	 0.2319986877264455
Epoch 8
Train acc and loss	 0.93325 	 0.20749437421560288
Val acc and loss	 0.92 	 0.2435502142761834
Epoch 9
Train acc and loss	 0.936 	 0.1929440341889858
Val acc and loss	 0.95 	 0.16520010924432427
Epoch 10
Train acc and loss	 0.94225 	 0.16198640383780002
Val acc and loss	 0.946 	 0

In [47]:
with open('trainvalAccs/' + sfile + '.txt', 'w') as f:
    for i in range(epochs):
        f.write("Epoch " + str(i + 1) + "\n")
        f.write("Train acc and loss\t" + str(train_accuracies[i]) + "\t" + str(train_losses[i]) + "\n")
        f.write("Val acc and loss\t" + str(val_accuracies[i]) + "\t" + str(val_losses[i]) + "\n")

In [26]:
# Save the model
torch.save(model.state_dict(), 'testsave/'+ sfile + '.pth')

In [49]:
# Layers of the model
model_layers = [16, "M", 32, "M", 64,"M"]
# Create model
model = small3DNet(model_layers, model_layers[-2], linear_neurons)
# model = separateChannel3DNet(model_layers, 16)
# # Load model
model.load_state_dict(torch.load('model_saves/new_fair/'+ sfile + '.pth'))
# Put model on gpu
model.cuda()
hi = 5

In [5]:
# Testing
    
wrong_dict = {}
right_dict = {}

for i in range(10):
    wrong_dict[i] = {}
    for j in range(10):
        wrong_dict[i][j] = 0
    right_dict[i] = 0
    
for it in range(1):
    # Total and amount correct
    test_correct = 0
    test_total = 0

    # Put the model in evaluation mode
    model.eval()

    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in tqdm(test_dataloader):
            # # Add color to each image
            # for i in range(len(images)):
            #     if 10 not in color_dict:
            #         colorize(images[i], labels[i].item(), color_dict)
            #     else:
            #         colorize_gaussian(images[i], labels[i].item(), color_dict)
            #     # images[i] = inv_normalize(images[i])

            # Add extra dimension for the network
            images = images.unsqueeze(1)

            # print(images.shape)

            # Put images
            images = images.cuda()

            # Predicted labels
            preds = model(images)

            # Top predictions per image
            _, top_preds = torch.max(preds, 1)

            # Predictions and images back on cpu
            top_preds = top_preds.cpu()
            images = images.cpu()
            
            # Check the predicted
            for i in range(len(labels)):
                if top_preds[i].item() == labels[i].item():
                    right_dict[top_preds[i].item()] += 1
                else:
                    wrong_dict[labels[i].item()][top_preds[i].item()] += 1

            # Amount of correct predictions
            predictions = [top_preds[i].item() == labels[i].item() for i in range(len(labels))]
            correct = np.sum(predictions)
            
            # if np.sum(predictions) < len(labels):
            #     print("hi")
            #     images = images.squeeze(1)
            #     # Show batch images
            #     # fig, axs = plt.subplots(4,8, figsize=(28, 28), facecolor='w', edgecolor='k')
            #     # fig.subplots_adjust(hspace = .5, wspace=.001)
            #     # axs = axs.ravel()
            #     # for i in range(len(images)):
            #     #     axs[i].imshow(images[i].permute(1, 2, 0))
                
                
#                 index = predictions.index(0)
                
#                 plt.imshow(images[index].permute(1, 2, 0))
#                 plt.show()
                
#                 print(index, "True:", labels[index].item(), "False:", top_preds[index].item())
                # break

            # Update total correct and total images
            test_correct += correct
            test_total += len(images)


    print("Correct", test_correct, "/", test_total, "Accuracy:", test_correct / test_total)
    

100%|██████████| 313/313 [00:01<00:00, 227.77it/s]

Correct 9656 / 10000 Accuracy: 0.9656


In [ ]:
# for x in right_dict:
    print(x, ":", right_dict[x])

In [ ]:
hm = []
for i, x in enumerate(wrong_dict):
    hm.append([])
    for y in wrong_dict[x]:
        hm[i].append(wrong_dict[x][y])
    # print(x, ":", wrong_dict[x])
print(hm)

In [ ]:
sns.heatmap(hm, cmap="Blues")